Этот код работает с тем же тестовым датасетом, что и исходный Time series Cashbacks, однако здесь задача состоит не в тесте, а в получении категорий на следующих месяц.

# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

import tensorflow_addons as tfa
import tensorflow as tf

from sklearn.preprocessing import StandardScaler

C:\Users\User\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
loaded_model = load_model("spendings.h5")

In [ ]:
best_look_back = 22

# Создание функций

## Создание временных фичей

In [ ]:
def add_time_features(df):
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['season'] = (df['month'] % 12 + 3) // 3 # 1: зима, 2: весна, 3: лето, 4: осень
    return df

## Создание датафрейма

Здесь мы учитываем, что если у клиента нет трат в течение последних двух лет, то пропуски заполняются нулями или меданами. Если у клиента нет трат вообще, то он получает случайных набор кэшбеков с небольшим разросом

In [ ]:
def get_dataframe(data):
    topics = ['автозапчасти', 'аквариум', 'видеоигры', 'закуски и приправы', 'напитки', 'образование',
              'одежда', 'продукты питания', 'уборка', 'электроника']

    data['date'] = pd.to_datetime(data['date'])
    new_data = add_time_features(data)

    data_grouped = new_data.groupby(['client', 'year', 'month', 'season', 'topic']).agg({'price': 'sum'}).reset_index()
    data_grouped = data_grouped.pivot_table(index=['year', 'month', 'season'], columns='topic', values='price', fill_value=0).reset_index(drop=True)

    data_grouped = data_grouped.reindex(columns=topics, fill_value=0)

    num_rows_needed = best_look_back - len(data_grouped)

    if num_rows_needed > 0:
        if len(data_grouped) > 0:
            median_values = data_grouped.median()
            for _ in range(num_rows_needed):
                data_grouped = pd.concat([pd.DataFrame([median_values], columns=topics), data_grouped]).reset_index(drop=True)
        else:
            random_values = {topic: np.random.randint(100, 200) for topic in topics}
            for _ in range(num_rows_needed):
                data_grouped = pd.concat([pd.DataFrame([random_values], columns=topics), data_grouped]).reset_index(drop=True)

    return data_grouped[topics]

## Получение списка кэшбеков

In [ ]:
def cashbaks_for_user(data):

    categories = pd.DataFrame()
    topics = ['автозапчасти', 'аквариум', 'видеоигры', 'закуски и приправы', 'напитки', 'образование',
              'одежда', 'продукты питания', 'уборка', 'электроника']

    df = get_dataframe(data)
    print(df)

    scaler = StandardScaler().fit(df.values)
    final_scaled_train = scaler.transform(df.values)

    # Построение и обучение модели с лучшими параметрами на всем тренировочном наборе данных
    model = loaded_model
    x_test = final_scaled_train[-best_look_back:].reshape(1, best_look_back, -1)
    # 4. Тестирование модели на тестовом наборе данных
    predictions = model.predict(x_test)

    predictions_original = scaler.inverse_transform(predictions)

    for index in range(len(topics)):
        categories.loc[index, 'topics'] = topics[index]
        categories.loc[index, 'predictions'] = predictions_original[0][index]

    print(categories)

    top = categories.sort_values(by='predictions', ascending=False).head(5).reset_index(drop=True)

    top.loc[0, 'percent'] = 10
    top.loc[4, 'percent'] = 3

    # Вычислить минимальное и максимальное значения
    min_val = top.loc[4, 'predictions']
    max_val = top.loc[0, 'predictions']

    # Рассчитать пропорциональные значения для 2, 3 и 4 мест
    for i in range(1, 4):
        proportion = (top.loc[i, 'predictions'] - min_val) / (max_val - min_val)
        top.loc[i, 'percent'] = round(3 + proportion * (10 - 3))

    cashbacks = top[['topics', 'percent']]

    return cashbacks

# Загрузка данных и получение прогноза

In [ ]:
df = pd.read_csv('test_df.csv')

In [ ]:
df

,Unnamed: 0,Столбец1,sale,category,price,client,cleaned_sale,topic,Unnamed: 7,Категория,Встречаемость,Unnamed: 10,Unnamed: 11,Unnamed: 12,date,quarter,month,year,season
0,0,29573,Видеокарта gigabyte geforce rtx 4070 aorus mas...,1227447308,75499,Объединенный_Клиент_1,видеокарта gigabyte geforce rtx aorus master g gv,электроника,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-06 05:29:04.852867,1,2,2023,1
1,1,29574,Видеокарта gigabyte geforce rtx 4060 eagle oc ...,1227447308,40999,Объединенный_Клиент_1,видеокарта gigabyte geforce rtx eagle oc gv,электроника,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-06 05:29:04.852867,1,2,2023,1
2,2,73799,Сало копченое бахрушинъ,1363498391,729,Объединенный_Клиент_1,сало копчёный бахрушинъ,продукты питания,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-12 08:46:28.852867,2,6,2023,3
3,3,73800,Пряники о'кей глазированные мятные 360 г,1363498391,60,Объединенный_Клиент_1,пряник глазированные мятный г,продукты питания,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-10 15:34:57.852867,3,7,2022,3
4,4,73801,Пирожное mirel тирамису 280 г,1363498391,219,Объединенный_Клиент_1,пирожное mirel тирамису г,продукты питания,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-30 10:22:51.852867,1,3,2023,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,715,251683,Салатная заправка я люблю готовить итальянский...,1964648565,154,Объединенный_Клиент_1,салатный заправка я любить готовить итальянски...,закуски и приправы,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-26 04:25:02.852867,2,6,2022,3
716,716,296570,Чай черный greenfield earl grey fantasy листов...,1976872004,189,Объединенный_Клиент_1,чай чёрный greenfield earl grey fantasy листов...,напитки,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-22 02:35:59.852867,4,11,2022,4
717,717,296571,"Газированный напиток добрый cola без сахара 0,...",1976872004,69,Объединенный_Клиент_1,газированный напиток добрый cola без сахар л,напитки,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-22 02:35:59.852867,4,11,2022,4
718,718,296572,Кофе santa ricci espresso italiano арабика-роб...,1976872004,1,Объединенный_Клиент_1,кофе santa ricci espresso italiano арабика роб...,напитки,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-22 02:35:59.852867,4,11,2022,4


In [ ]:
cashbacks = cashbaks_for_user(df)

topic  автозапчасти  аквариум  видеоигры  закуски и приправы  напитки  \
0               678         0        398                  19      870   
1              3708      1120          0                 340     1182   
2              1491      7258          0                1238      129   
3                 0         0       3548                1197      584   
4                 0      6541        899                 996     1145   
5              7722      3255        668                 737     1432   
6              1904      3067          0                  88      959   
7              3719         0        349                 611     1366   
8              5188         0       1198                 647     1280   
9               345      4311          0                 491      273   
10              565      2615          0                 763        0   
11                0      1811        615                 115     1098   
12            16696      1928        549           

In [ ]:
cashbacks

,topics,percent
0,электроника,10.0
1,аквариум,3.0
2,продукты питания,3.0
3,образование,3.0
4,напитки,3.0
